# Comparison Methods
Generating asv_biomarker and person_biomarker matrices, and biomarker names for SBB1s, ASVs, OTUs, Taxa, and Micropheno biomarkers.

In [1]:
import sys
#sys.path.append('/oak/stanford/groups/dpwall/computeEnvironments/micropheno/MicroPheno/')
#from make_representations.representation_maker import Metagenomic16SRepresentation, FastaRepresentations
import numpy as np
import pandas as pd
from Bio import SeqIO
import scipy.sparse
from scipy.spatial.distance import jaccard
from tqdm import tqdm
BIOMARKER_DIR = '/home/groups/dpwall/briannac/sequence_based_biomarkers/results/generate_biomarkers/'

In [2]:
sys.path.append('/oak/stanford/groups/dpwall/computeEnvironments/ditaxa/DiTaxa/')
from main.DiTaxa import DiTaxaWorkflow

/share/software/user/open/py-scikit-learn/0.19.1_py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [99]:
def sparse_unique_columns(M):
    M = M.tocsc()
    m, n = M.shape
    if not M.has_sorted_indices:
        M.sort_indices()
    if not M.has_canonical_format:
        M.sum_duplicates()
    sizes = np.diff(M.indptr)
    idx = np.argsort(sizes)
    Ms = M@scipy.sparse.csc_matrix((np.ones((n,)), idx, np.arange(n+1)), (n, n))
    ssizes = np.diff(Ms.indptr)
    ssizes[1:] -= ssizes[:-1]
    grpidx, = np.where(ssizes)
    grpidx = np.concatenate([grpidx, [n]])
    if ssizes[0] == 0:
        counts = [np.array([0, grpidx[0]])]
    else:
        counts = [np.zeros((1,), int)]
    ssizes = ssizes[grpidx[:-1]].cumsum()
    for i, ss in tqdm(enumerate(ssizes)):
        gil, gir = grpidx[i:i+2]
        pl, pr = Ms.indptr[[gil, gir]]
        dv = Ms.data[pl:pr].view(f'V{ss*Ms.data.dtype.itemsize}')
        iv = Ms.indices[pl:pr].view(f'V{ss*Ms.indices.dtype.itemsize}')
        idxi = np.lexsort((dv, iv))
        dv = dv[idxi]
        iv = iv[idxi]
        chng, = np.where(np.concatenate(
            [[True], (dv[1:] != dv[:-1]) | (iv[1:] != iv[:-1]), [True]]))
        counts.append(np.diff(chng))
        idx[gil:gir] = idx[gil:gir][idxi]
    counts = np.concatenate(counts)
    nu = counts.size - 1
    uniques = M@scipy.sparse.csc_matrix((np.ones((nu,)), idx[counts[:-1].cumsum()],
                                   np.arange(nu + 1)), (n, nu))
    return uniques, idx[counts[:-1].cumsum()], counts[1:]


In [100]:
# Compute person x biomarker matrix
def SaveMatrices(asv_biomarker, biomarkers, dataset, biomarker_type):
    sample_vs_asv = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/sample_vs_asv.tsv' % dataset.replace('testing_', '').replace('small_', ''), index_col=0)
    person_biomarker = scipy.sparse.csc_matrix(sample_vs_asv.transpose())*asv_biomarker
    biomarker_exists  = [True for b in biomarkers]
        
    _, no_ld_idx, _ = sparse_unique_columns(asv_biomarker)
    no_ld_idx = sorted(no_ld_idx)

    print("Number of biomarkers after LD filtering: ", len(no_ld_idx))

    #np.save('%sperson_variant%d_condensed' %  (output_dir,order), person_biomarker)
    print('Computing biomarker names...')

    #asv_biomarker = scipy.sparse.csr_matrix(asv_biomarker[:,no_ld_idx])
    person_biomarker = scipy.sparse.csr_matrix(person_biomarker[:,no_ld_idx])
    biomarkers = [biomarkers[i] for i in no_ld_idx]
    is_abundant = ((person_biomarker>0).mean(axis=0)>.1).tolist()[0]
    print(sum(is_abundant), ' biomarkers > 10% freq')
    asv_biomarker = asv_biomarker[:,np.where(is_abundant)[0]]
    person_biomarker = person_biomarker[:,np.where(is_abundant)[0]]
    biomarkers = [biomarkers[i] for i in np.where(is_abundant)[0]]    
    unique_idx_current = 0
    current_idx = 0
    with open(BIOMARKER_DIR + 'biomarker_names_%s_%s.txt' % (biomarker_type, dataset), 'w') as f:
        for i,biomarker in tqdm(enumerate(biomarkers)):
            #if biomarker_exists[i]:  # If biomarker doesn't even exist, just skip.
            #    if unique_idx_current==len(no_ld_idx):
            #        break
            #    elif current_idx==no_ld_idx[unique_idx_current]:
            #        if is_abundant[unique_idx_current]: 
            f.write(str(biomarker) + '\n')
            #        unique_idx_current += 1
            #    current_idx += 1
    print("Writing to files...")
    scipy.sparse.save_npz(BIOMARKER_DIR + 'asv_vs_biomarker_%s_%s.npz' % (biomarker_type, dataset), asv_biomarker)  # Save sparse matrix of ASV x biomarker.
    scipy.sparse.save_npz(BIOMARKER_DIR + 'sample_vs_biomarker_%s_%s.npz' % (biomarker_type, dataset), person_biomarker)  # Save sparse matrix of ASV x biomarker.


# DiTaxa

In [105]:
for dataset in ['obesity', 'autism']:
    print('processing ', dataset)
    P = DiTaxaWorkflow('/scratch/users/briannac/tmp/ditaxa/' + dataset,
                       'fa','/scratch/users/briannac/tmp/ditaxa/' + dataset,dataset,50000,5000,-1,'',num_p=20)
    P.train_npe()
    P.representation_npe()

    with open('/scratch/users/briannac/tmp/ditaxa/%s/intermediate_files/npe_representation/%s_uniquepiece_-1_features' % (dataset, dataset)) as f:
        features = [ff.replace('\n', '').upper() for ff in f.readlines()]
    seqs = [str(r.seq) for r in SeqIO.parse('/scratch/users/briannac/tmp/ditaxa/%s/seqs.fa' % dataset, 'fasta')]
    asv_biomarker = scipy.sparse.csc_matrix([[1*(f in s) for f in features] for s in seqs])
    SaveMatrices(asv_biomarker, [i for i in range(np.shape(asv_biomarker)[1])], dataset, 'ditaxa')

processing  obesity
1  fasta files found in /scratch/users/briannac/tmp/ditaxa/obesity
	✔ DiTaxa workflow is getting started
	✔ Segmentation inference started.. 
Segmentation training
1 fasta files found in /scratch/users/briannac/tmp/ditaxa/obesity


100%|██████████| 1/1 [00:00<00:00,  8.38it/s]


	✔ Corpus size for training NPE is  5000
	✔ The segmentation training took  31.656883273040876  ms.
	✔ Creating NPE representations ...
1 fasta files found in /scratch/users/briannac/tmp/ditaxa/obesity


100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


	✔ Generating the NPE representations at npe_representation/obesity_uniquepiece_-1  2.445842742919922 seconds , using 20cores


...

	✔ Corpus size for training NPE is  5000
	✔ The segmentation training took  49.07554459897801  ms.
	✔ Creating NPE representations ...
1 fasta files found in /scratch/users/briannac/tmp/ditaxa/autism


100%|██████████| 1/1 [00:00<00:00,  1.12it/s]


	✔ Generating the NPE representations at npe_representation/autism_uniquepiece_-1  1.5521552562713623 seconds , using 20cores


1168it [00:00, 23844.84it/s]
11880it [00:00, 893990.20it/s]


Number of biomarkers after LD filtering:  21234
Computing biomarker names...
11880  biomarkers > 10% freq
Writing to files...


# Micropheno

In [9]:
for dataset in ['autism', 'obesity']:
    for k in [4,6,8]:
        seqs_file='/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/seqs.fa' % dataset
        
        FastaRep=FastaRepresentations(seqs_file,label_modifying_func=lambda x: x.split('.')[0])
        asv_biomarker=FastaRep.get_vector_rep(FastaRep.corpus, k,restricted=True)
        
        SaveMatrices(asv_biomarker, [i for i in range(np.shape(asv_biomarker)[1])], dataset, 'micropheno%s' % k)

244it [00:00, 30581.51it/s]
256it [00:00, 682607.64it/s]


Number of biomarkers after LD filtering:  256
Computing biomarker names...
256  biomarkers > 10% freq
Writing to files...


832it [00:00, 31722.17it/s]
3595it [00:00, 885669.48it/s]


Number of biomarkers after LD filtering:  4053
Computing biomarker names...
3595  biomarkers > 10% freq
Writing to files...


775it [00:00, 23738.16it/s]
9356it [00:00, 907075.68it/s]


...

Number of biomarkers after LD filtering:  256
Computing biomarker names...
256  biomarkers > 10% freq
Writing to files...


904it [00:00, 31059.76it/s]
2855it [00:00, 876627.96it/s]


Number of biomarkers after LD filtering:  3962
Computing biomarker names...
2855  biomarkers > 10% freq
Writing to files...


904it [00:00, 25664.52it/s]
4878it [00:00, 885361.33it/s]


Number of biomarkers after LD filtering:  13350
Computing biomarker names...
4878  biomarkers > 10% freq
Writing to files...


# 1 SBB

In [10]:
for dataset in ['autism', 'obesity']:
    rdp = [record for record in SeqIO.parse('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/seqs_aligned_rdp.fa' % dataset, 'fasta')][:-1]
    df = pd.DataFrame([list(r.seq.upper()) for r in rdp])
    is_interesting = df.apply(lambda x: (len(set(x))>0) & (len(set(x).intersection(['A', 'T', 'C', 'G']))>0))

    df = df[df.columns[is_interesting]]
    df_A = df=='A'
    df_A.columns = [str(c) + '_A' for c in df_A.columns]
    df_T = df=='T'
    df_T.columns = [str(c) + '_T' for c in df_T.columns]
    df_G = df=='G'
    df_G.columns = [str(c) + '_G' for c in df_G.columns]
    df_C = df=='C'
    df_C.columns = [str(c) + '_C' for c in df_C.columns]
    df_gap = df=='-'
    df_gap.columns = [str(c) + '_gap' for c in df_gap.columns]
    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]

    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)

    # Get rid of biomarkers at 100% frequency, 0% frequency, and in LD w/another.
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]

    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(df_sbb1)
    SaveMatrices(asv_biomarker, df_sbb1.columns, dataset, 'sbb1')

488it [00:00, 29453.62it/s]
644it [00:00, 868251.94it/s]


Number of biomarkers after LD filtering:  1121
Computing biomarker names...
644  biomarkers > 10% freq
Writing to files...


498it [00:00, 28806.56it/s]
594it [00:00, 857335.37it/s]


Number of biomarkers after LD filtering:  912
Computing biomarker names...
594  biomarkers > 10% freq
Writing to files...


# 1SBB small

In [28]:
for dataset in ['autism_small', 'obesity_small']:
    rdp = [record for record in SeqIO.parse('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/seqs_aligned_rdp.fa' % dataset, 'fasta')][:-1]
    df = pd.DataFrame([list(r.seq.upper()) for r in rdp])
    is_interesting = df.apply(lambda x: (len(set(x))>0) & (len(set(x).intersection(['A', 'T', 'C', 'G']))>0))

    df = df[df.columns[is_interesting]]
    df_A = df=='A'
    df_A.columns = [str(c) + '_A' for c in df_A.columns]
    df_T = df=='T'
    df_T.columns = [str(c) + '_T' for c in df_T.columns]
    df_G = df=='G'
    df_G.columns = [str(c) + '_G' for c in df_G.columns]
    df_C = df=='C'
    df_C.columns = [str(c) + '_C' for c in df_C.columns]
    df_gap = df=='-'
    df_gap.columns = [str(c) + '_gap' for c in df_gap.columns]
    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]

    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)

    # Get rid of biomarkers at 100% frequency, 0% frequency, and in LD w/another.
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]

    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(df_sbb1)
    
    asv_vs_biomarker_df = pd.DataFrame(asv_biomarker.todense())
    asv_vs_biomarker_df.columns = df_sbb1.columns
    df = asv_vs_biomarker_df[asv_vs_biomarker_df.columns[(asv_vs_biomarker_df.mean()>.1) & (asv_vs_biomarker_df.mean()<.9)]]
    
    jaccard_df = pd.DataFrame([[jaccard(df[df.columns[i]], df[df.columns[j]]) if i > j else np.nan 
                            for i in range(np.shape(df)[1])] 
                           for j in range(np.shape(df)[1])])
    new_df = jaccard_df
    min_val = np.min(new_df.min(axis=0))
    min_idx = new_df.columns[np.argmin(new_df.min(axis=0))]
    while (min_val < .1) or (min_val>.9):
        new_df = new_df.drop(min_idx, axis=0)
        new_df = new_df.drop(min_idx, axis=1)
        min_val = np.min(new_df.min(axis=0))
        min_idx = new_df.columns[np.argmin(new_df.min(axis=0))]
    columns = df.columns[new_df.columns]
    print(len(columns))
    asv_biomarker = scipy.sparse.csc_matrix(df[columns])
    SaveMatrices(asv_biomarker, columns, dataset, 'sbb1')

197


188it [00:00, 31183.18it/s]
197it [00:00, 436721.93it/s]


Number of biomarkers after LD filtering:  197
Computing biomarker names...
197  biomarkers > 10% freq
Writing to files...
149


147it [00:00, 29028.38it/s]
149it [00:00, 410881.85it/s]


Number of biomarkers after LD filtering:  149
Computing biomarker names...
149  biomarkers > 10% freq
Writing to files...


# 1 SBB for testing

In [12]:
for dataset in ['testing_autism', 'testing_obesity']:
    rdp = [record for record in SeqIO.parse('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/seqs_aligned_rdp.fa' % dataset.replace('testing_', ''), 'fasta')][:-1]
    df = pd.DataFrame([list(r.seq.upper()) for r in rdp])
    is_interesting = df.apply(lambda x: (len(set(x))>0) & (len(set(x).intersection(['A', 'T', 'C', 'G']))>0))

    df = df[df.columns[is_interesting]]
    df_A = df=='A'
    df_A.columns = [str(c) + '_A' for c in df_A.columns]
    df_T = df=='T'
    df_T.columns = [str(c) + '_T' for c in df_T.columns]
    df_G = df=='G'
    df_G.columns = [str(c) + '_G' for c in df_G.columns]
    df_C = df=='C'
    df_C.columns = [str(c) + '_C' for c in df_C.columns]
    df_gap = df=='-'
    df_gap.columns = [str(c) + '_gap' for c in df_gap.columns]
    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]
    df_sbb1 = pd.concat([df_A, df_C, df_T, df_G, df_gap], axis=1)

    # Get rid of biomarkers at 100% frequency, 0% frequency, and in LD w/another.
    df_sbb1 = df_sbb1[df_sbb1.columns[(df_sbb1.mean()>0.0) & (df_sbb1.mean()<1.0)]]
    df_sbb1 = df_sbb1[df_sbb1.columns[~df_sbb1.transpose().duplicated().values]]
    df_sbb1 = df_sbb1[df_sbb1.columns[::20]] # JUST 100 COLUMNS FOR TESTING
    print(np.shape(df_sbb1))
    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(df_sbb1)
    SaveMatrices(asv_biomarker, df_sbb1.columns, dataset, 'sbb1')

(5249, 57)


43it [00:00, 28505.62it/s]
57it [00:00, 323511.95it/s]


Number of biomarkers after LD filtering:  57
Computing biomarker names...
34  biomarkers > 10% freq
Writing to files...
(12363, 46)


44it [00:00, 27970.50it/s]
46it [00:00, 281250.71it/s]

Number of biomarkers after LD filtering:  46
Computing biomarker names...
37  biomarkers > 10% freq
Writing to files...


# OTUs

In [12]:
for dataset in ['autism', 'obesity']:
    for similarity in [90, 95, 97, 99]:
        otu_mapping = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/results/generate_biomarkers/otu_asv_map_%i_%s.txt' % (similarity, dataset), header=None)
        otu_dict = {otu:idx for idx, otu in enumerate(np.unique(otu_mapping[1]))}
        otu_mapping[1] = [otu_dict[i] for i in otu_mapping[1]]
        mat = np.zeros((len(otu_mapping), len(otu_dict)))
        for i, otu_id in enumerate(otu_mapping[1]):
            mat[i][otu_id] = 1
        asv_biomarker = scipy.sparse.csc_matrix(mat)
        SaveMatrices(asv_biomarker, list(otu_dict.keys()), dataset, 'otu%s' % similarity)

59it [00:00, 30300.47it/s]
181it [00:00, 423643.43it/s]


Number of biomarkers after LD filtering:  506
Computing biomarker names...
181  biomarkers > 10% freq
Writing to files...


45it [00:00, 26982.66it/s]
297it [00:00, 464469.91it/s]


Number of biomarkers after LD filtering:  1334
Computing biomarker names...
297  biomarkers > 10% freq
Writing to files...


33it [00:00, 23114.90it/s]
365it [00:00, 473090.53it/s]


...

Number of biomarkers after LD filtering:  2077
Computing biomarker names...
218  biomarkers > 10% freq
Writing to files...


50it [00:00, 22047.43it/s]
283it [00:00, 462943.85it/s]


Number of biomarkers after LD filtering:  3809
Computing biomarker names...
283  biomarkers > 10% freq
Writing to files...


14it [00:00, 3657.22it/s]
288it [00:00, 458951.20it/s]

Number of biomarkers after LD filtering:  9525
Computing biomarker names...
288  biomarkers > 10% freq
Writing to files...


# ASVs

In [13]:
for dataset in ['autism', 'obesity']:
    sample_vs_asv = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/sample_vs_asv.tsv' % dataset, index_col=0)

    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(np.identity(len(sample_vs_asv)))
    SaveMatrices(asv_biomarker, sample_vs_asv.index, dataset, 'asv')

1it [00:00, 964.65it/s]
416it [00:00, 482530.55it/s]


Number of biomarkers after LD filtering:  5249
Computing biomarker names...
416  biomarkers > 10% freq
Writing to files...


1it [00:00, 474.47it/s]
278it [00:00, 461788.72it/s]

Number of biomarkers after LD filtering:  12363
Computing biomarker names...
278  biomarkers > 10% freq
Writing to files...


# Taxa

In [14]:
for dataset in ['autism', 'obesity']:
    annotations = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/asv_vs_taxa_annotation.tsv' % dataset, index_col=0)
    sample_vs_asv = pd.read_table('/home/groups/dpwall/briannac/sequence_based_biomarkers/data/%s/sample_vs_asv.tsv' % dataset, index_col=0)
    annotations['Kingdom'] = ['k_' +a for a in annotations['Kingdom']]
    annotations['Phylum'] = ['p_' +a for a in annotations['Phylum']]
    annotations['Class'] = ['c_' +a for a in annotations['Class']]
    annotations['Order'] = ['o_' +a for a in annotations['Order']]
    annotations['Family'] = ['f_' +a for a in annotations['Family']]
    annotations['Genus'] = ['g_' +a for a in annotations['Genus']]
    taxa = np.unique(np.concatenate(annotations.values))
    taxa_df = pd.DataFrame(np.zeros((len(annotations), len(taxa))))
    taxa_df.columns = taxa
    taxa_df.index = annotations.index
    taxa_df = taxa_df[[c for c in taxa_df.columns if 'unclassified' not in c]]
    for row in annotations.iterrows():
        taxa_df.loc[row[0],list([r for r in row[1].values if 'unclassified' not in r])]  = 1
        
        
    # Save sparse matrix of ASV x biomarker.
    asv_biomarker = scipy.sparse.csc_matrix(taxa_df)
    SaveMatrices(asv_biomarker, taxa_df.columns, dataset, 'taxa')

73it [00:00, 30719.79it/s]
183it [00:00, 670356.01it/s]


Number of biomarkers after LD filtering:  358
Computing biomarker names...
183  biomarkers > 10% freq
Writing to files...


86it [00:00, 30964.90it/s]
101it [00:00, 541027.72it/s]

Number of biomarkers after LD filtering:  192
Computing biomarker names...
101  biomarkers > 10% freq
Writing to files...
